In [1]:
#https://www.tensorflow.org/tutorials/keras/text_classification_with_hub


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [3]:
# DL IMDB dataset

In [4]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

In [5]:
# binary/two class classification
    # training and testing set are equal in size = thus, balanced test/train split

In [6]:
# format:
    # each unit/example is a sentence representing the movie review + a corresponding label
        # label is 0 or 1; 0 is negative review, 1 is positive
# print the first ten examples:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=219, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [7]:
# print first ten labels
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [8]:
# NN ---> stacked layers
    # how to represent the text?
    # how many layers to use in the model?
    # how many hidden units to use for each layer?

# input data ----> sentences, with labels/target to predict of 0 or 1
    # converting sentences into embedding vectors (one way to represent the text)

In [9]:
# create keras layer that uses a pre-trained TF hub model to embed the sentences and test on a few examples:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
    # pre trained model from tensorflow hub
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=402, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [10]:
# build full model:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [11]:
# configure model to use optimizer and loss function 
    # binary_crossentropy ----> binary classification problem and model outputs a probability 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
# train model for 20 epochs, in mini batches of 512 samples
    # monitor model loss/accuracy on the the validation set while training

In [13]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================].7508 - accuracy: 0.54 - 1s 621ms/step - loss: 0.7886 - accuracy: 0.509 - 1s 437ms/step - loss: 0.7983 - accuracy: 0.492 - 1s 344ms/step - loss: 0.7998 - accuracy: 0.484 - 1s 288ms/step - loss: 0.7911 - accuracy: 0.488 - 1s 250ms/step - loss: 0.7832 - accuracy: 0.498 - 2s 222ms/step - loss: 0.7770 - accuracy: 0.498 - 2s 201ms/step - loss: 0.7733 - accuracy: 0.495 - 2s 185ms/step - loss: 0.7648 - accuracy: 0.502 - 2s 172ms/step - loss: 0.7606 - accuracy: 0.501 - 2s 160ms/step - loss: 0.7562 - accuracy: 0.503 - 2s 151ms/step - loss: 0.7548 - accuracy: 0.499 - 2s 143ms/step - loss: 0.7506 - accuracy: 0.501 - 2s 136ms/step - loss: 0.7475 - accuracy: 0.500 - 2s 130ms/step - loss: 0.7439 - accuracy: 0.501 - 2s 125ms/step - loss: 0.7405 - accuracy: 0.504 - 2s 121ms/step - loss: 0.7375 - accuracy: 0.505 - 2s 116ms/step - loss: 0.7345 - accuracy: 0.508 - 2s 113ms/step - loss: 0.7318 - accuracy: 0.510 - 2s 110ms/step - loss: 0.7287 - accuracy: 0.5

In [14]:
#evaluate the model
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3279 - accuracy: 0.8580
loss: 0.328
accuracy: 0.858
